In [1]:
import pandas as pd

import numpy as np
from scipy.stats import invgamma, norm
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import statsmodels.api as sm

from pathlib import Path
import glob
import os

In [2]:
def exponential_model(t, beta, gamma, t0):
    return beta * np.exp(gamma * (t - t0))

def compute_log_likelihood(y, t, beta, gamma, sigma, t0):
    y_pred = exponential_model(t, beta, gamma, t0)
    return np.sum(norm.logpdf(y, loc=y_pred, scale=sigma))

def gibbs_sample_sigma(y, beta, gamma, t, t0, alpha1=4, alpha2=1):
    residuals = y - exponential_model(t, beta, gamma, t0)
    N = len(y)
    alpha_post = N / 2 + alpha1
    beta_post = np.sum(residuals**2) / 2 + alpha2
    return invgamma.rvs(a=alpha_post, scale=beta_post)

def metropolis_hastings(y, t, t0=0, n_samples=5000, burn_in=500, thin=5,
                        beta_range=(0.01, 10), gamma_range=(-1, 1), proposal_sd=(0.1, 0.05)):

    samples = []
    beta, gamma = 1.0, 0.0
    sigma = 1.0

    for i in range(n_samples):
        beta_prop = np.clip(np.random.normal(beta, proposal_sd[0]), *beta_range)
        gamma_prop = np.clip(np.random.normal(gamma, proposal_sd[1]), *gamma_range)

        loglike_current = compute_log_likelihood(y, t, beta, gamma, sigma, t0)
        loglike_prop = compute_log_likelihood(y, t, beta_prop, gamma_prop, sigma, t0)

        accept_prob = np.exp(loglike_prop - loglike_current)
        if np.random.rand() < accept_prob:
            beta, gamma = beta_prop, gamma_prop

        sigma = gibbs_sample_sigma(y, beta, gamma, t, t0)

        samples.append((beta, gamma, sigma))

    # Discard burn-in and thin
    samples = samples[burn_in::thin]
    return np.array(samples)


In [3]:
def normalize_series(y):
    y = (y - np.min(y)) / (np.max(y) - np.min(y) + 1e-8)
    return y + 1  # Set minimum to 1

def sliding_window_pvalues(y, window=14):
    pvalues = []
    t = np.arange(len(y))
    y_norm = normalize_series(y)

    for start in range(len(y) - window + 1):
        t_win = t[start:start+window]
        y_win = y_norm[start:start+window]

        samples = metropolis_hastings(y_win, t_win, t0=t_win[0])
        gamma_samples = samples[:, 1]
        pval = 1 - np.mean(gamma_samples > 0)
        pvalues.append(pval)

    return np.array(pvalues)


# Run model for OTC and APS series

In [4]:
mun_new = list(Path('/opt/storage/refined/aesop/visualization/')
               .glob('aesop_*_mun_new.parquet'))

aesop_mun_new = max(mun_new, key=lambda x: x.stat().st_ctime)

df = pd.read_parquet(aesop_mun_new)

In [10]:
dta_model = df[['co_ibge', 'year_week', 'epidemi_cal_start', 'atend_ivas', 'num_otc_ivas']] 

In [34]:
lst = []

for code in dta_model.co_ibge.unique():

    print(code)

    set_muni = dta_model[dta_model.co_ibge == code]
    set_muni = set_muni.assign(ma_atend_ivas = set_muni.atend_ivas.rolling(window=8).mean().fillna(0),
                          ma_otc_ivas =  set_muni.num_otc_ivas.rolling(window=8).mean().fillna(0))

    pvals_aps_ivas = sliding_window_pvalues(set_muni[-30:].atend_ivas.to_numpy(), window=8)

    pvals_otc_ivas = sliding_window_pvalues(set_muni[-30:].num_otc_ivas.to_numpy(), window=8)

    set_muni2 = set_muni[-23:].copy()

    set_muni2 = set_muni2.assign(P_growth_aps_ivas = 1 - pvals_aps_ivas,
                            P_growth_otc_ivas = 1 - pvals_otc_ivas)

    lst.append(set_muni2)

110001
110002
110003
110004
110005
110006
110007
110008
110009
110010
110011
110012
110013
110014
110015
110018
110020
110025
110026
110028
110029
110030
110032
110033
110034
110037
110040
110045
110070
110080
110090
110092
110094
110100
110110
110120
110130
110140
110143
110145
110146
110147
110148
110149
110150
110155
110160
110170
110175
110180
120001
120005
120010
120013
120017
120020
120025
120030
120032
120033
120034
120035
120038
120039
120040
120042
120043
120045
120050
120060
120070
120080
130002
130006
130008
130010
130014
130020
130030
130040
130050
130060
130063
130068
130070
130080
130083
130090
130100
130110
130115
130120
130130
130140
130150
130160
130165
130170
130180
130185
130190
130195
130200
130210
130220
130230
130240
130250
130255
130260
130270
130280
130290
130300
130310
130320
130330
130340
130350
130353
130356
130360
130370
130380
130390
130395
130400
130406
130410
130420
130423
130426
130430
130440
140002
140005
140010
140015
140017
140020
140023
140028
140030

ValueError: Domain error in arguments. The `scale` parameter must be positive for all distributions, and many distributions have restrictions on shape parameters. Please see the `scipy.stats.invgamma` documentation for details.